#### Let's Begin by importing both the libs sklearn and cuml to measure the performance difference

Starting with the basics lets try Random Forest Classifier for checking heardisease

In [ ]:
!nvidia-smi

In [ ]:
import cudf as df
import pandas as pd

## Let's import the data now

In [ ]:
data_df=df.read_csv("heart-disease.csv")
data_pd=pd.read_csv("heart-disease.csv")

In [ ]:
data_df

### getting training and test data split ready 

In [ ]:
from sklearn.model_selection import train_test_split as tts_sk
from cuml.model_selection import train_test_split as tts_ml

In [ ]:
x=data_df.drop('target',axis=1)
y=data_df['target']

X=data_pd.drop('target',axis=1)
Y=data_pd['target']

In [ ]:
x_train,x_test,y_train,y_test = tts_ml(x,y,test_size=0.2)

In [ ]:
X_train,X_test,Y_train,Y_test = tts_sk(X,Y,test_size=0.2)

### Importing Models 

In [ ]:
from cuml.ensemble import RandomForestClassifier as rfc_ml
from sklearn.ensemble import RandomForestClassifier as rfc_sk

### Fittin Data

In [ ]:
clf_ml=rfc_ml()
clf_sk=rfc_sk()

In [ ]:
from cuml.model_selection import GridSearchCV as GridSearchCV_ml
from sklearn.model_selection import GridSearchCV as GridSearchCV_sk


import cupy as py 
import numpy as np

### Trying Models CUML
 

In [ ]:
clf_ml.fit(x_train,y_train)

In [ ]:
clf_ml.score(x_test,y_test)

### Trying Models Sklearn


In [ ]:
clf_sk.fit(X_train,Y_train)

In [ ]:
clf_sk.score(X_test,Y_test)

In [ ]:
param_grid = {'n_estimators': [i for i in range(10, 100, 1)]}

# Setup the grid search
grid = GridSearchCV_ml(clf_ml,
                    param_grid,
                    cv=5,
                    verbose=True)

# Fit the grid search to the data
grid.fit(X_train, Y_train)

# Find the best parameters
grid.best_params_

In [ ]:
param_grid = {'n_estimators': [i for i in range(10, 100, 1)]}

# Setup the grid search
grid = GridSearchCV_ml(clf_sk,
                    param_grid,
                    cv=5,
                    verbose=True,
                    n_jobs=-1)

# Fit the grid search to the data
grid.fit(X_train, Y_train)

# Find the best parameters
grid.best_params_

## Time to Test Bigger Data

### Bulldoser Data 

Note: this notebook from further on will only contain CuML code. If you're intersted in Sklearn look at the the bulldoser.ipynb in the same repo

In [1]:
import cudf as df
from cuml.model_selection import train_test_split
import cupy as py 

In [2]:
dt=df.read_csv("bluebook-for-bulldozers/TrainAndValid.csv",parse_dates=['saledate'])

In [3]:
dt

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3,2004,68,Low,2006-11-16,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
1,1139248,57000.0,117657,77,121,3,1996,4640,Low,2004-03-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3,2001,2838,High,2004-02-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,1139251,38500.0,1026470,332,121,3,2001,3486,High,2011-05-19,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1139253,11000.0,1057373,17311,121,3,2007,722,Medium,2009-07-23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412693,6333344,10000.0,1919201,21435,149,2,2005,<NA>,<NA>,2012-03-07,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412694,6333345,10500.0,1882122,21436,149,2,2005,<NA>,<NA>,2012-01-28,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412695,6333347,12500.0,1944213,21435,149,2,2005,<NA>,<NA>,2012-01-28,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412696,6333348,10000.0,1794518,21435,149,2,2006,<NA>,<NA>,2012-03-07,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>


In [4]:
dt.isnull().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

In [5]:
len(dt)

412698

In [6]:
dt.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   SalesID                   412698 non-null  int64
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64
 3   ModelID                   412698 non-null  int64
 4   datasource                412698 non-null  int64
 5   auctioneerID              392562 non-null  int64
 6   YearMade                  412698 non-null  int64
 7   MachineHoursCurrentMeter  147504 non-null  int64
 8   UsageBand                 73670 non-null   object
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object
 11  fiBaseModel               412698 non-null  object
 12  fiSecondaryDesc           271971 non-null  object
 13  fiModelSeries             58667 non-null   object
 14  fi

In [7]:
dt=dt.sort_values(by="saledate")

In [8]:
dt

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
205615,1646770,9500.0,1126363,8434,132,18,1974,<NA>,<NA>,1989-01-17,...,<NA>,<NA>,<NA>,<NA>,<NA>,None or Unspecified,Straight,None or Unspecified,<NA>,<NA>
28603,1259551,9500.0,1292226,6788,132,18,1977,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28757,1259760,8000.0,1476205,6788,132,99,1978,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28820,1259842,9000.0,1226323,6788,132,18,1978,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28927,1259973,7700.0,1150485,6788,132,18,1978,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412164,6318085,5500.0,1873607,10402,149,99,2001,<NA>,<NA>,2012-04-28,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
412218,6319596,20500.0,1887135,18110,149,99,2005,<NA>,<NA>,2012-04-28,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
412436,6321656,14000.0,1823084,18631,149,99,2006,<NA>,<NA>,2012-04-28,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412476,6324811,6000.0,1919104,19330,149,99,2004,<NA>,<NA>,2012-04-28,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
dt["saleDay"]=dt.saledate.dt.day
dt["saleMonth"]=dt.saledate.dt.month
dt["saleYear"]=dt.saledate.dt.year

dt["saleDayofweek"] = dt.saledate.dt.dayofweek
dt["saleDayofyear"] = dt.saledate.dt.dayofyear

In [10]:
dt=dt.drop("saledate",axis=1)

In [11]:
dic=dt.items()

for column,data in dic:
    if pd.api.types.is_object_dtype(data):
        dt[column]=data.astype('category').cat.as_ordered()

NameError: name 'pd' is not defined

### The Above code is producing error as the size is diffrent. Check thread
 https://stackoverflow.com/questions/42382263/valueerror-length-of-values-does-not-match-length-of-index-pandas-dataframe-u

### Initial Solution is too fill all the nan values with something 

### Lets Try that 

In [ ]:
# dic=dt.items()

# for column,data in dic:
#     if pd.api.types.is_object_dtype(data):
#         dt[column].fillna("Empty")

In [12]:
dt.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 412698 entries, 205615 to 412544
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   SalesID                   412698 non-null  int64
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64
 3   ModelID                   412698 non-null  int64
 4   datasource                412698 non-null  int64
 5   auctioneerID              392562 non-null  int64
 6   YearMade                  412698 non-null  int64
 7   MachineHoursCurrentMeter  147504 non-null  int64
 8   UsageBand                 73670 non-null   object
 9   fiModelDesc               412698 non-null  object
 10  fiBaseModel               412698 non-null  object
 11  fiSecondaryDesc           271971 non-null  object
 12  fiModelSeries             58667 non-null   object
 13  fiModelDescriptor         74816 non-null   object
 14  Produ

In [13]:
dt.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension             386715
Blade_Width         

In [14]:
dt.UsageBand.count()

73670

In [ ]:
dt["UsageBand"]=dt.UsageBand.fillna(py.nan)

In [ ]:
dt.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                        0
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

In [16]:
dic=dt.items()

for column,data in dic:
    if df.api.types.is_object_dtype(data):
        dt[column]=dt[column].fillna("Missing")

In [17]:
dt.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                        0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc                  0
fiModelSeries                    0
fiModelDescriptor                0
ProductSize                      0
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                     0
Enclosure                        0
Forks                            0
Pad_Type                         0
Ride_Control                     0
Stick                            0
Transmission                     0
Turbocharged                     0
Blade_Extension                  0
Blade_Width         

In [18]:
dt["auctioneerID"]=dt["auctioneerID"].fillna(-1)

In [19]:
dt["MachineHoursCurrentMeter"]=dt["MachineHoursCurrentMeter"].fillna(-1)

In [20]:
dt.isna().sum()

SalesID                     0
SalePrice                   0
MachineID                   0
ModelID                     0
datasource                  0
auctioneerID                0
YearMade                    0
MachineHoursCurrentMeter    0
UsageBand                   0
fiModelDesc                 0
fiBaseModel                 0
fiSecondaryDesc             0
fiModelSeries               0
fiModelDescriptor           0
ProductSize                 0
fiProductClassDesc          0
state                       0
ProductGroup                0
ProductGroupDesc            0
Drive_System                0
Enclosure                   0
Forks                       0
Pad_Type                    0
Ride_Control                0
Stick                       0
Transmission                0
Turbocharged                0
Blade_Extension             0
Blade_Width                 0
Enclosure_Type              0
Engine_Horsepower           0
Hydraulics                  0
Pushblock                   0
Ripper    

In [26]:
dt[column]

205615    Missing
28603     Missing
28757     Missing
28820     Missing
28927     Missing
           ...   
412164    Missing
412218    Missing
412436    Missing
412476    Missing
412544    Missing
Name: UsageBand, Length: 412698, dtype: category
Categories (4, object): ['High' < 'Low' < 'Medium' < 'Missing']

In [27]:
dt.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 412698 entries, 205615 to 412544
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   SalesID                   412698 non-null  int64
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64
 3   ModelID                   412698 non-null  int64
 4   datasource                412698 non-null  int64
 5   auctioneerID              412698 non-null  int64
 6   YearMade                  412698 non-null  int64
 7   MachineHoursCurrentMeter  412698 non-null  int64
 8   UsageBand                 412698 non-null  category
 9   fiModelDesc               412698 non-null  object
 10  fiBaseModel               412698 non-null  object
 11  fiSecondaryDesc           412698 non-null  object
 12  fiModelSeries             412698 non-null  object
 13  fiModelDescriptor         412698 non-null  object
 14  Pro

In [23]:
column="UsageBand"
dt[column]=data.astype('category').cat.as_ordered()

ValueError: Length of values (206349) does not match length of index (412698)

In [28]:
dic=dt.items()

for column,data in dic:
    if df.api.types.is_object_dtype(data):
        try:
            dt[column]=data.astype('category').cat.as_ordered()
        except:
            print("error")

error
error


In [29]:
dt.info()


<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 412698 entries, 205615 to 412544
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   SalesID                   412698 non-null  int64
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64
 3   ModelID                   412698 non-null  int64
 4   datasource                412698 non-null  int64
 5   auctioneerID              412698 non-null  int64
 6   YearMade                  412698 non-null  int64
 7   MachineHoursCurrentMeter  412698 non-null  int64
 8   UsageBand                 412698 non-null  category
 9   fiModelDesc               412698 non-null  object
 10  fiBaseModel               412698 non-null  object
 11  fiSecondaryDesc           412698 non-null  category
 12  fiModelSeries             412698 non-null  category
 13  fiModelDescriptor         412698 non-null  category
 1

In [33]:
len(dt.fiModelDesc)

412698

In [34]:
dt["fiModelDesc"]=data.astype('category').cat.as_ordered()

ValueError: Length of values (206349) does not match length of index (412698)

In [35]:
dic=dt.items()

for column,data in dic:
    if column=="fiModelDesc":
        dt["fiModelDesc"]=df.Series(data)
        print("Done")

Done


In [37]:
dt=dt.drop("fiModelDesc",axis=1)

In [38]:
dt=dt.drop("fiBaseModel",axis=1)

In [39]:
dt.isna().sum()

SalesID                     0
SalePrice                   0
MachineID                   0
ModelID                     0
datasource                  0
auctioneerID                0
YearMade                    0
MachineHoursCurrentMeter    0
UsageBand                   0
fiSecondaryDesc             0
fiModelSeries               0
fiModelDescriptor           0
ProductSize                 0
fiProductClassDesc          0
state                       0
ProductGroup                0
ProductGroupDesc            0
Drive_System                0
Enclosure                   0
Forks                       0
Pad_Type                    0
Ride_Control                0
Stick                       0
Transmission                0
Turbocharged                0
Blade_Extension             0
Blade_Width                 0
Enclosure_Type              0
Engine_Horsepower           0
Hydraulics                  0
Pushblock                   0
Ripper                      0
Scarifier                   0
Tip_Contro

## Attempt to get data ready for machine learning unsuccefull using CUDF too many errors

### Lets Try a new approach 

Editing Data with pandas and converting it to CuDF dataframe

In [40]:
import pandas as pd
import cupy as py
import cudf as df

In [41]:
dt=pd.read_csv("bluebook-for-bulldozers/TrainAndValid.csv",low_memory=False,parse_dates=['saledate'])

In [42]:
dt.sort_values(by='saledate',inplace=True,ascending=True)

In [43]:
dt_copy=dt.copy()

In [44]:
dt["saleDay"]=dt.saledate.dt.day
dt["saleMonth"]=dt.saledate.dt.month
dt["saleYear"]=dt.saledate.dt.year

dt["saleDayofweek"] = dt.saledate.dt.dayofweek
dt["saleDayofyear"] = dt.saledate.dt.dayofyear

# Drop original saledate
dt.drop("saledate", axis=1, inplace=True)

In [45]:
dic=dt.items()

for column,data in dic:
    if pd.api.types.is_object_dtype(data):
        dt[column]=data.astype('category').cat.as_ordered()

In [46]:
dt.to_csv("bluebook-for-bulldozers/train_tmp.csv",
              index=False)

In [48]:
df_tmp = pd.read_csv("bluebook-for-bulldozers/train_tmp.csv",
                     low_memory=False)
df_tmp.head().T

,0,1,2,3,4
SalesID,1646770,1821514,1505138,1671174,1329056
SalePrice,9500.0,14000.0,50000.0,16000.0,22000.0
MachineID,1126363,1194089,1473654,1327630,1336053
ModelID,8434,10150,4139,8591,4089
datasource,132,132,132,132,132
auctioneerID,18.0,99.0,99.0,99.0,99.0
YearMade,1974,1980,1978,1980,1984
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
fiModelDesc,TD20,A66,D7G,A62,D3B


In [49]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

auctioneerID
MachineHoursCurrentMeter


In [50]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # Add a binary column which tells if the data was missing our not
            df_tmp[label+"_is_missing"] = pd.isnull(content)
            # Fill missing numeric values with median since it's more robust than the mean
            df_tmp[label] = content.fillna(content.median())

In [51]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [52]:
df_tmp.auctioneerID_is_missing.value_counts()

False    392562
True      20136
Name: auctioneerID_is_missing, dtype: int64

In [53]:
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

UsageBand
fiModelDesc
fiBaseModel
fiSecondaryDesc
fiModelSeries
fiModelDescriptor
ProductSize
fiProductClassDesc
state
ProductGroup
ProductGroupDesc
Drive_System
Enclosure
Forks
Pad_Type
Ride_Control
Stick
Transmission
Turbocharged
Blade_Extension
Blade_Width
Enclosure_Type
Engine_Horsepower
Hydraulics
Pushblock
Ripper
Scarifier
Tip_Control
Tire_Size
Coupler
Coupler_System
Grouser_Tracks
Hydraulics_Flow
Track_Type
Undercarriage_Pad_Width
Stick_Length
Thumb
Pattern_Changer
Grouser_Type
Backhoe_Mounting
Blade_Type
Travel_Controls
Differential_Type
Steering_Controls


In [54]:
for label, content in df_tmp.items():
    # Check columns which *aren't* numeric
    if not pd.api.types.is_numeric_dtype(content):
        # Add binary column to inidicate whether sample had missing value
        df_tmp[label+"_is_missing"] = pd.isnull(content)
        # We add the +1 because pandas encodes missing categories as -1
        df_tmp[label] = pd.Categorical(content).codes+1        

In [56]:
df_temp=df.from_pandas(df_tmp,nan_as_null=True)

In [57]:
%%time

from cuml.ensemble import RandomForestRegressor
# Instantiate model
model = RandomForestRegressor(verbose=True)

# Fit the model
model.fit(df_temp.drop("SalePrice", axis=1), df_temp.SalePrice)

CPU times: user 1min 46s, sys: 1min 47s, total: 3min 33s
Wall time: 1min 11s


/home/yashdhingra/miniconda3/envs/MLenv2/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


RandomForestRegressor()